# Lightweight Fine-Tuning Project

TODO: In this cell, describe your choices for each of the following

* **PEFT technique:** Sequence to classification task type is used to fine tume with PEFT
* **Model:** "cardiffnlp/twitter-roberta-base-sentiment-latest"
* **Evaluation approach:** Accuracy
* **Fine-tuning dataset:** https://huggingface.co/datasets/financial_phrasebank
 Whose label values are 0: negative, 1: neutral, 3: positive

This report demonstrates how a twitter-based sentiment analysis model is applied to financial comment sentiment analysis. 

To adapt the original model to apply to the financial domain model, PEFT as  Fine-Tuning approach will be taken. To compare with the Full Fine-Tuning result, Full Fine-Tunins is also evaluated.

In [1]:
# utility
from IPython.display import display, Markdown

GPU is enabled for training.

In [53]:
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

## Loading and Evaluating a Foundation Model

In the cells below, load your chosen pre-trained Hugging Face model and evaluate its performance prior to fine-tuning. This step includes loading an appropriate tokenizer and dataset.

### Loading Dataset

In the cell below, financial sentiment dataset is loaded, and split them to training and test dataset with 8:2 proportion.

In [3]:
# Load the financial_phrasebank dataset
# See: https://huggingface.co/datasets/financial_phrasebank
# Whose label values are 0: negative, 1: neutral, 3: positive

from datasets import load_dataset

# The sms_spam dataset only has a train split, so we use the train_test_split method to split it into train and test
dataset = load_dataset("financial_phrasebank", "sentences_50agree", split="train").train_test_split(
    test_size=0.2, shuffle=True, seed=23
)

splits = ["train", "test"]

# View the dataset characteristics
dataset

DatasetDict({
    train: Dataset({
        features: ['sentence', 'label'],
        num_rows: 3876
    })
    test: Dataset({
        features: ['sentence', 'label'],
        num_rows: 970
    })
})

In [4]:
display(Markdown("#### sample data set"))
display(dataset["train"].to_pandas().head(10))


#### sample data set

,sentence,label
0,"Nokia bought Chicago-based Navteq in 2008 , ac...",2
1,The deal will have no significant effect on th...,1
2,"Loudeye Corp. , up $ 2.56 at $ 4.33 Nokia Corp...",1
3,7 March 2011 - Finnish IT company Digia Oyj HE...,2
4,"The total number of voting rights is 74,612,523 .",1
5,Amanda Capital has investments in 22 private e...,1
6,One attractive factor about the tablets for me...,2
7,The implementation of the deal is subject to t...,1
8,As part of the reorganisation measures that wi...,0
9,"As part of the reorganisation , Pohjola Insura...",1


As a utility, label and index matching is defined here.

In [5]:
id2label={0: "negative", 1: "neutral", 2: "positive"}
label2id={"negative": 0, "neutral": 1, "positive": 2}

Model and tokenizer is loaded in the cell below.

In [6]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer, AutoConfig
import torch

MODEL = f"cardiffnlp/twitter-roberta-base-sentitokenizer = AutoTokenizer.from_pretrained(MODEL)
config = AutoConfig.from_pretrained(MODEL)

# Pretrained Model
model = AutoModelForSequenceClassification.from_pretrained(
    MODEL,
    num_labels=3,
    id2label=id2label,
    label2id=label2id,
).to(device)

Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Evaluate the pre-trained model peformance with accuracy for tha financial dataset.

In [27]:
sentiment = ["negative", "neutral", "positive"]

def get_prediction(comment):
    """Given a comment, return the predicted sentiment"""

    # Tokenize the review
    # (Get the response as tensors and not as a list)
    inputs = tokenizer(comment, return_tensors="pt").to(device)

    # Perform the prediction (get the logits)
    outputs = model(**inputs)

    predictions = torch.argmax(outputs.logits, dim=-1)

    return predictions.item()

In [8]:
accurate_count = 0
test_size = len(dataset["test"])
for a in dataset["test"]:
    if a["label"] == get_prediction(a["sentence"]):
        accurate_count += 1

display(Markdown(f"**Pre Fine-Tuning Accuracy :** {accurate_count / test_size}"))

**Pre Fine-Tuning Accuracy :** 0.6824742268041237

## Performing Full Fine-Tuning

To compare PEFT approach, run the full tuning.

In [9]:
# Unfreeze all the model parameters.
# Hint: Check the documentation at https://huggingface.co/transformers/v4.2.2/training.html
for param in model.parameters():
    param.require_grad = True

model

RobertaForSequenceClassification(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0): RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerN

In [10]:
# Let's use a lambda function to tokenize all the examples
tokenized_dataset = {}
for split in splits:
    tokenized_dataset[split] = dataset[split].map(
        lambda x: tokenizer(x["sentence"], truncation=True), batched=True
    )

# Inspect the available columns in the dataset
tokenized_dataset

Map:   0%|          | 0/970 [00:00<?, ? examples/s]Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Map: 100%|██████████| 970/970 [00:00<00:00, 13918.45 examples/s]


{'train': Dataset({
     features: ['sentence', 'label', 'input_ids', 'attention_mask'],
     num_rows: 3876
 }),
 'test': Dataset({
     features: ['sentence', 'label', 'input_ids', 'attention_mask'],
     num_rows: 970
 })}

In [11]:
import numpy as np
from transformers import DataCollatorWithPadding, Trainer, TrainingArguments


def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return {"accuracy": (predictions == labels).mean()}


# The HuggingFace Trainer class handles the training and eval loop for PyTorch for us.
# Read more about it here https://huggingface.co/docs/transformers/main_classes/trainer
trainer = Trainer(
    model=model,
    args=TrainingArguments(
        output_dir="./data/sentiment_analysys",
        learning_rate=2e-5,
        per_device_train_batch_size=16,
        per_device_eval_batch_size=16,
        evaluation_strategy="epoch",
        save_strategy="epoch",
        num_train_epochs=6, # Throuch some experiences, this value is most accurate.
        weight_decay=0.01,
        load_best_model_at_end=True,
    ),
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    tokenizer=tokenizer,
    data_collator=DataCollatorWithPadding(tokenizer=tokenizer),
    compute_metrics=compute_metrics,
)

trainer.train()

  0%|          | 0/1458 [00:00<?, ?it/s]You're using a RobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
                                                  
 17%|█▋        | 243/1458 [01:18<05:02,  4.02it/s]

{'eval_loss': 0.36716505885124207, 'eval_accuracy': 0.8474226804123711, 'eval_runtime': 4.4917, 'eval_samples_per_second': 215.953, 'eval_steps_per_second': 13.581, 'epoch': 1.0}


                                                  
 33%|███▎      | 486/1458 [02:38<03:57,  4.09it/s]

{'eval_loss': 0.41259732842445374, 'eval_accuracy': 0.8597938144329897, 'eval_runtime': 4.4835, 'eval_samples_per_second': 216.348, 'eval_steps_per_second': 13.605, 'epoch': 2.0}


 34%|███▍      | 500/1458 [02:48<05:23,  2.96it/s]

...

                                                   
100%|██████████| 1458/1458 [08:07<00:00,  4.00it/s]

{'eval_loss': 0.8098055720329285, 'eval_accuracy': 0.8752577319587629, 'eval_runtime': 4.4843, 'eval_samples_per_second': 216.308, 'eval_steps_per_second': 13.603, 'epoch': 6.0}


100%|██████████| 1458/1458 [08:12<00:00,  2.96it/s]

{'train_runtime': 492.8841, 'train_samples_per_second': 47.184, 'train_steps_per_second': 2.958, 'train_loss': 0.16481368662725587, 'epoch': 6.0}


TrainOutput(global_step=1458, training_loss=0.16481368662725587, metrics={'train_runtime': 492.8841, 'train_samples_per_second': 47.184, 'train_steps_per_second': 2.958, 'train_loss': 0.16481368662725587, 'epoch': 6.0})

After a full fine-tuning, we have a model with 88% accuracy. 

## Performing Parameter-Efficient Fine-Tuning

TODO: In the cells below, create a PEFT model from your loaded model, run a training loop, and save the PEFT model weights.

In [7]:
LORA_MODEL_NAME = "sentiment-lora"

In [8]:
config

RobertaConfig {
  "_name_or_path": "cardiffnlp/twitter-roberta-base-sentiment-latest",
  "architectures": [
    "RobertaForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "negative",
    "1": "neutral",
    "2": "positive"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "negative": 0,
    "neutral": 1,
    "positive": 2
  },
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "torch_dtype": "float32",
  "transformers_version": "4.33.3",
  "type_vocab_size": 1,
  "use_cache": true,
  "vocab_size": 50265
}

In [35]:
from peft import LoraConfig, TaskType, get_peft_model

lora_config = LoraConfig(
    task_type=TaskType.SEQ_CLS,
    inference_mode=False,
    r=32,
    lora_alpha=32,
    lora_dropout=0.1
    )

In [36]:
peft_model = get_peft_model(model, lora_config)
peft_model.print_trainable_parameters()

trainable params: 1,772,547 || all params: 126,420,486 || trainable%: 1.4021042444022878


In [37]:
# Let's use a lambda function to tokenize all the examples
tokenized_dataset = {}
for split in splits:
    tokenized_dataset[split] = dataset[split].map(
        lambda x: tokenizer(x["sentence"], padding=True, truncation=True, return_tensors="pt"), batched=True
    )

# Inspect the available columns in the dataset
tokenized_dataset

Map: 100%|██████████| 3876/3876 [00:00<00:00, 21079.97 examples/s]


{'train': Dataset({
     features: ['sentence', 'label', 'input_ids', 'attention_mask'],
     num_rows: 3876
 }),
 'test': Dataset({
     features: ['sentence', 'label', 'input_ids', 'attention_mask'],
     num_rows: 970
 })}

In [33]:
print(tokenized_dataset["train"][0]["sentence"])
print(tokenizer.decode(tokenized_dataset["train"][0]["input_ids"]))

Nokia bought Chicago-based Navteq in 2008 , acquiring a maps database to compete with Google s maps as well as with navigation device companies such as TomTom NV and Garmin Ltd. .
<s>Nokia bought Chicago-based Navteq in 2008, acquiring a maps database to compete with Google s maps as well as with navigation device companies such as TomTom NV and Garmin Ltd..</s><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad>


In [38]:
import numpy as np
from transformers import DataCollatorWithPadding, Trainer, TrainingArguments


def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return {"accuracy": (predictions == labels).mean()}


# The HuggingFace Trainer class handles the training and eval loop for PyTorch for us.
# Read more about it here https://huggingface.co/docs/transformers/main_classes/trainer
trainer = Trainer(
    model=peft_model,
    args=TrainingArguments(
        output_dir="./data/" + LORA_MODEL_NAME,
        learning_rate=2e-5,
        per_device_train_batch_size=16,
        per_device_eval_batch_size=16,
        evaluation_strategy="epoch",
        save_strategy="epoch",
        num_train_epochs=6, # Throuch some experiences, this value is most accurate.
        weight_decay=0.01,
        load_best_model_at_end=True,
    ),
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    tokenizer=tokenizer,
    data_collator=DataCollatorWithPadding(tokenizer=tokenizer),
    compute_metrics=compute_metrics,
)

trainer.train()

  0%|          | 0/1458 [56:02<?, ?it/s]
                                                  
 17%|█▋        | 243/1458 [01:22<05:48,  3.49it/s]

{'eval_loss': 0.44943967461586, 'eval_accuracy': 0.8061855670103093, 'eval_runtime': 10.3802, 'eval_samples_per_second': 93.447, 'eval_steps_per_second': 5.877, 'epoch': 1.0}


                                                  
 33%|███▎      | 486/1458 [02:42<04:37,  3.50it/s]

{'eval_loss': 0.409159392118454, 'eval_accuracy': 0.8247422680412371, 'eval_runtime': 10.2936, 'eval_samples_per_second': 94.233, 'eval_steps_per_second': 5.926, 'epoch': 2.0}


 34%|███▍      | 500/1458 [02:47<05:04,  3.14it/s]

...

                                                   
100%|██████████| 1458/1458 [08:15<00:00,  3.78it/s]

{'eval_loss': 0.37303289771080017, 'eval_accuracy': 0.8371134020618557, 'eval_runtime': 10.8942, 'eval_samples_per_second': 89.038, 'eval_steps_per_second': 5.599, 'epoch': 6.0}


100%|██████████| 1458/1458 [08:16<00:00,  2.94it/s]

{'train_runtime': 496.3019, 'train_samples_per_second': 46.859, 'train_steps_per_second': 2.938, 'train_loss': 0.4339911797082637, 'epoch': 6.0}


TrainOutput(global_step=1458, training_loss=0.4339911797082637, metrics={'train_runtime': 496.3019, 'train_samples_per_second': 46.859, 'train_steps_per_second': 2.938, 'train_loss': 0.4339911797082637, 'epoch': 6.0})

In [45]:
peft_model

PeftModelForSequenceClassification(
  (base_model): LoraModel(
    (model): RobertaForSequenceClassification(
      (roberta): RobertaModel(
        (embeddings): RobertaEmbeddings(
          (word_embeddings): Embedding(50265, 768, padding_idx=1)
          (position_embeddings): Embedding(514, 768, padding_idx=1)
          (token_type_embeddings): Embedding(1, 768)
          (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (encoder): RobertaEncoder(
          (layer): ModuleList(
            (0): RobertaLayer(
              (attention): RobertaAttention(
                (self): RobertaSelfAttention(
                  (query): lora.Linear(
                    (base_layer): Linear(in_features=768, out_features=768, bias=True)
                    (lora_dropout): ModuleDict(
                      (default): Dropout(p=0.1, inplace=False)
                    )
                    (lora_A): ModuleDict(
   

## Performing Inference with a PEFT Model

TODO: In the cells below, load the saved PEFT model weights and evaluate the performance of the trained PEFT model. Be sure to compare the results to the results from prior to fine-tuning.

In [50]:
from peft import PeftModelForSequenceClassification
saved_model = PeftModelForSequenceClassification.from_pretrained(model, LORA_MODEL_NAME)

In [51]:
accurate_count = 0
count = 0
test_size = len(dataset["test"])
for a in dataset["test"]:
    inputs = tokenizer(a["sentence"], return_tensors="pt").to(device)
    outputs = saved_model(inputs["input_ids"])
    predictions = torch.argmax(outputs.logits, dim=-1)
    if a["label"] == predictions.item():
        accurate_count += 1

display(Markdown(f"**PEFT Accuracy:** {accurate_count / test_size}"))

**PEFT Accuracy:** 0.8391752577319588

## Summary

This report demonstrates how a twitter-based sentiment analysis model is applied to financial comment sentiment analysis. As anticipated, the model performance enhanced after some fine-tuning. The accuracy of PEFT (Parameter-Efficient Fine-Tuning) is inferior to Full Fine-Tuning, but increased to 84 % from pre-trained model accuracy 68%.

The duration for Full Fine-Tuning and PEFT Accuracy is comparable, but I have detected through OS monitoring, less RAM is utilized in PEFT model Fine-Tuning.

Full Fine-Tuning nearly exhausted the RAM resource in my PC, while PEFT consumed less than 12GB in this case.



| Tuning           | Accuracy Output    |Time to Fine-Tune |
|------------------|--------------------|--------|
| Pre Fine-Tuning  | 0.6824742268041237 |
| Full Fine-Tuning | 0.8752577319587629 | 8min 16sec|
| PEFT Accuracy    | 0.8391752577319588 | 8 min 12 sec 